# 1. Documentation

## Version 1:
- Program base (voice activation and audio response)
- Integration of IMDb functions

## Libraries

- SpeechRecognition: to recognize audio input
- PyAudio: for audio I/O
- Requests: to send http requests
- Playsound: to play audio files (not necessary)
- gTTS: for voice recognition and audio response on the robot's side
- IMDbPy: for imdb search queries
- json: to parse json files

In [1]:
import speech_recognition as sr
from IPython.display import *
from gtts import gTTS
import os
from playsound import playsound
import datetime
from datetime import date
import webbrowser
import time
import json
import requests
import imdb
import random
import urllib3

In [2]:
urllib3.disable_warnings()

# remove mp3 files if exist
if os.path.isfile('ready.mp3'):
    os.remove("ready.mp3")

if os.path.isfile('response.mp3'):
    os.remove("response.mp3")
    
if os.path.isfile('hello.mp3'):
    os.remove("hello.mp3")

# 2. Functions

### Basic functions

In [3]:
def error_message():
    tts = gTTS(text='Ich habe Sie nicht richtig verstanden. Beginnen wir noch mal von vorne.', lang='de')
    
    # check if the file already exists, if so just play it (if not save it and then play it)
    if os.path.isfile('error.mp3'):
        playsound("error.mp3")
    else:
        tts.save("error.mp3")
        playsound("error.mp3")
    
    print("ERROR: Ihr Audio-Guide ist verwirrt!")

In [4]:
def welcome():
    tts = gTTS(text='Herzlich willkommen! Mein Name ist Alex und ich bin Ihr Audio-Guide. In kürze können Sie sich über die jeweiligen Stationen informieren.', lang='de')
    
    # check if the file already exists, if so just play it (if not save it and then play it)
    if os.path.isfile('hello.mp3'):
        playsound("hello.mp3")
    else:
        tts.save("hello.mp3")
        playsound("hello.mp3")

In [5]:
def ready():
    tts = gTTS(text='Wie kann ich Ihnen behilflich sein?', lang='de')
    
    # check if the file already exists, if so just play it (if not save it and then play it)
    if os.path.isfile('ready.mp3'):
        playsound("ready.mp3")
    else:
        tts.save("ready.mp3")
        playsound("ready.mp3")

### IMDb functions
The functions that are called when the user wants to learn something about movies

In [6]:
# get informations about a specific movie
def imdb_search(this_movie):
    print("\nSuche nach Film: {}".format(this_movie))
    
    #start imdb search -> problem: if there are multiple movies with the same title it picks the first one
    i_movie = imdb.IMDb()
    movies = i_movie.search_movie(this_movie)
    movie = i_movie.get_movie(movies[0].movieID)
    
    print("Ich habe folgenden Film gefunden: {}\n".format(movies[0]))
    response = "Ich habe den Film" + str(movies[0]) + "gefunden"
    tts = gTTS(response, lang='de')
                    
    print("DEBUG 13: Deleting previous responses")
    if os.path.isfile('response.mp3'):
        os.remove("response.mp3")

    tts.save("response.mp3")
    playsound("response.mp3")
    
    try:
        print("\nGenres:\n")
        for genre in movie['genres']:
            print(genre)
    except KeyError:
        print("\nKein Genre bekannt")
    
    try:
        # print the plot
        print("\nHandlung:\n{}".format(movie['plot'][0]))
    except KeyError:
        print("\nKeine Handlung bekannt")
    
    try:
        # print the cast
        print("\nSchauspieler:\n")
        for actor in movie['cast'][:5]: #looks at the first five actors
            print("{0} as {1}".format(actor['name'], actor.currentRole))
    except KeyError:
        print("\nKeine Schauspieler bekannt")
    
    try:
        # print the director
        if movie['directors'] is not None:
            print("\nRegisseur: {}\n".format(movie['director'][0]))
    except KeyError:
        print("\nKein Regisseur bekannt")

    try:
        # print the release date
        print("\nErscheinungsdatum: {}\n".format(movie['original air date']))
    except KeyError:
        print("\nErscheinungstermin unbekannt")

        
#look for top 10 movies by a specific genre
def imdb_genre(this_genre):

    print("\nIch suche nach Filmen mit Genre '{}'. (Das kann etwas dauern)".format(this_genre))
    response = "Ich suche nach Filmen mit Genre" + str(this_genre)
    tts = gTTS(response, lang='de')
                    
    print("DEBUG 13: Deleting previous responses")
    if os.path.isfile('response.mp3'):
        os.remove("response.mp3")

    tts.save("response.mp3")
    playsound("response.mp3")
    
    #get a list of the top 250 movies
    i_movie = imdb.IMDb()
    movielist = i_movie.get_top250_movies()
    count = 0
    listed_movies = []
    while count < 10: 
        
        # search for new random number
        while True:
            ranking = random.randint(0, 249)
            if ranking not in listed_movies:
                listed_movies.append(ranking)
                break
        
        movie = i_movie.get_movie(movielist[ranking].movieID)
        for genre in movie['genres']:
            #check if th first genre matches the genre the user was looking for
            if genre.lower() == this_genre:
                count += 1
                print("{0}. {1}".format(count, movie['title']))
                
                tts = gTTS(movie['title'], lang='de')

                print("DEBUG 13: Deleting previous responses")
                if os.path.isfile('response.mp3'):
                    os.remove("response.mp3")

                tts.save("response.mp3")
                playsound("response.mp3")
                
    print("Das sollte fürs erste genügen :)")

### Intent pool
The program looks at the json file to check if a voice command fits the programs intents

In [7]:
def intents(req):
    # debugging:
    print("DEBUG 6: In intents function")
    print("DEBUG 7: Deleting previous responses")
    
    if os.path.isfile('response.mp3'):
        os.remove("response.mp3")
        
    # open json file
    with open('intents.json') as json_file:
        # load json file
        data = json.load(json_file)

        # debugging:
        print("DEBUG 8: Looking through intents in the json file")
        
        # go through intents
        for i in data['intents']:
            # find the right request
            for p in i['patterns']:
                if req == p.lower():
                    # debugging:
                    print("DEBUG 9: Request matches intent. Looking for response")
                    
                            
                    # pick a randomized response for that request
                    index = random.randint(0, len(i['responses'])-1)
                    
                    # debugging:
                    print("DEBUG 10: Found a response. About to send it to tts")
                    
                    response = i['responses'][index]
                    tts = gTTS(response, lang='de')
                    
                    # debugging:
                    print("DEBUG 11: This is the response i should give the user")
                    print(response)
                    
                    tts.save("response.mp3")
                    playsound("response.mp3")
                    
                    if i['tag'] == "imdb_search":
                        request = mic_recognition(rec, mic)
                        if request['error'] is None:
                            imdb_search(request['response_trans'].lower())
                    elif i['tag'] == "imdb_genre":
                        request = mic_recognition(rec, mic)
                        if request['error'] is None:
                            imdb_genre(request['response_trans'].lower())
                    # if the user wishes to exit the program
                    elif i['tag'] == "bye":
                        tts = gTTS(response, lang='de')
                        tts.save("response.mp3")
                        playsound("response.mp3")
                        return True
                    
                    return False
        
        print("Error: I understood '{}'".format(req))
        tts = gTTS('Verzeihung, aber ich verstehe Ihre Anfrage nicht. Probieren wir es nochmals von Anfang an!', lang='de')
                    
        print("DEBUG 13: Deleting previous responses")
        if os.path.isfile('response.mp3'):
            os.remove("response.mp3")
                        
        tts.save("response.mp3")
        playsound("response.mp3")
        
        return False

        

# 3. Speech recognition

### Audio/input parser
Record audio and send it to Google's API to parse/"translate"

In [8]:
#make a recognizer query while checking for different errors and return an according response 
def mic_recognition(recognizer, microphone):
    if not isinstance(recognizer, sr.Recognizer):
        raise TypeError("`recognizer` must be `Recognizer` instance")

    if not isinstance(microphone, sr.Microphone):
        raise TypeError("`microphone` must be `Microphone` instance")
        
    response = {
      "response_trans": None,
      "error": None
    }
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source, duration=0.5)
        audio_recorded = recognizer.listen(source)
    try:
        response["response_trans"] = recognizer.recognize_google(audio_recorded, language = "de-DE")
    except sr.RequestError:
        #API was unreachable or unresponsive
        response["error"] = "ERROR: Die API ist nicht verfügbar."
    except sr.UnknownValueError:
        #speech was unintelligible
        response["error"] = "ERROR: Der Audio-Guide konnte den Sprach-Input nicht verstehen."
        
    return response

### Speech recognition
Immediate interaction with the robot takes place here. The program takes the received input and checks if said input can be found in its intent pool

In [9]:
def activate_speechrec():
    # debugging:
    print("DEBUG 3: In activate_speechrec. Sending welcome message")
    
    welcome()
    while True:
        ready()
        
        # debugging:
        print("DEBUG 4: In activate_speechrec while loop. Ready for request.")
        
        request = mic_recognition(rec, mic)
        if request["error"] is None:
            
            # debugging:
            print("DEBUG 5: Got a request. Going into intents function")
            
            stop = intents(request["response_trans"].lower())
            if stop == True:
                break
        else:
            print(request["error"])
            error_message()

### Voice activation
Activating robot interaction starts with the phrase "hallo alex"

In [10]:
keyword = "hallo alex"
success = False

#Initialize recognizer instance and a microphone as a source
rec = sr.Recognizer()
mic = sr.Microphone()

# listen for keyword (or only listen a certain amount of times?)
while True:
    print("Ich bin bereit ...")
    request = mic_recognition(rec, mic)
    
    #check if voice was recognized
    if request["error"] is None:
        if request["response_trans"].lower() == keyword:
            success = True
            
            # debugging:
            print("DEBUG 1: Recognized keyword")
            break
        else:
            print("Error: I understood: {}".format(request["response_trans"].lower()))

#now you can speak to the bot
if success is True:
    
    # debugging:
    print("DEBUG 2: Entering activate_speechrec function")
    activate_speechrec()
else:
    print(request["error"])
    error_message()

Ich bin bereit ...
DEBUG 1: Recognized keyword
DEBUG 2: Entering activate_speechrec function
DEBUG 3: In activate_speechrec. Sending welcome message
DEBUG 4: In activate_speechrec while loop. Ready for request.
DEBUG 5: Got a request. Going into intents function
DEBUG 6: In intents function
DEBUG 7: Deleting previous responses
DEBUG 8: Looking through intents in the json file
DEBUG 9: Request matches intent. Looking for response
DEBUG 10: Found a response. About to send it to tts
DEBUG 11: This is the response i should give the user
Auf welches Genre hast du Lust?

Ich suche nach Filmen mit Genre 'drama'. (Das kann etwas dauern)
DEBUG 13: Deleting previous responses
1. Inside Out
DEBUG 13: Deleting previous responses
2. Django Unchained
DEBUG 13: Deleting previous responses
3. Mr. Smith Goes to Washington
DEBUG 13: Deleting previous responses
4. The Deer Hunter
DEBUG 13: Deleting previous responses
5. The Intouchables
DEBUG 13: Deleting previous responses
6. The Lives of Others
DEBUG 1

PermissionError: [Errno 13] Permission denied: 'response.mp3'